The purpose of this notebook is to illustrate how to upload data from the coleridge-initiative-data dataset quickly


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import re
import glob

def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
submission_df = pd.read_csv('../input/coleridge-initiative-data/submission.csv', index_col=0)
submission_df['PredictionString'] = None
indata = open('../input/coleridge-initiative-data/confirmed_databases.pickle','rb')
database_dict = pickle.load(indata)
indata.close()
datasets_titles = database_dict['original'] #Fetch the original set of databases (and labels) that we know about
#Test data has to be built
test_files = glob.glob("../input/coleridgeinitiative-show-us-the-data/test/*.json")
df_test_publications = pd.DataFrame()

for test_file in test_files:
    file_data = pd.read_json(test_file)
    file_data.insert(0,'pub_id', test_file.split('/')[-1].split('.')[0])
    df_test_publications = pd.concat([df_test_publications, file_data])

df_test_publications.to_csv("df_test_publications.csv",index=False)
df_test_publications['text'] = df_test_publications['text'].str.lower()
df_train_publications = pd.read_csv('../input/coleridge-initiative-data/df_train_publications.csv')

df_train_publications.head(3)

In [ ]:
df_test_publications.head(3)

In [ ]:
labels = []
for index in submission_df.index:
    print(index)
    publication_text = df_test_publications[df_test_publications['pub_id'] == index].text.str.cat(sep='\n').lower()
    label = []
    for dataset_title in datasets_titles:
        if dataset_title in publication_text:
            label.append(clean_text(dataset_title))
    labels.append('|'.join(label))

submission_df['PredictionString'] = labels

submission_df.to_csv('submission.csv')

submission_df